# Colab
This first block is only for starting in Colabratory. Some pips need to be re-installed each time and authentication codes need to be entered from links to allow access to google drive files. Kind of annoying but free GPU and such (be sure to tic it on in notebook settings under edit). Remove this block if not using Colabratory.

In [0]:
from google.colab import auth
auth.authenticate_user()
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p gdrive
!google-drive-ocamlfuse gdrive
!pip install -q keras
!pip install numba

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) 

    100% |████████████████████████████████| 15.8MB 3.0MB/s 


# Imports

In [0]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from keras.utils import np_utils

from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, Embedding, Flatten, Activation, SpatialDropout1D
from keras.layers import Bidirectional, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, MaxPooling1D, Conv1D
from keras.optimizers import Adam
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'

import re
import math
# set seed
np.random.seed(123)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


This block is to set up and/or alter GPU settings. Remove if not using GPU

In [0]:
# GPU acceleration
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"
from keras import backend as K
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.Session(config=config)

# Read in data
There are 4375 training examples, which is somewhat small for deep learning. However, there is lots of information to extract in the description and the smaller data set allows us to compensate by building a deeper model without becoming computationally infeasible.

In [0]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\n')
# tags = pd.read_csv('tags.tsv', sep='\t')

In [0]:
print(train.shape[0])
print(test.shape[0])
train[:20]

4375
2921


,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...
5,associate-needed 5-plus-years-experience-needed,"At Cottage Health System, our facilities are s..."
6,full-time-job ms-or-phd-needed 5-plus-years-ex...,*E Learning Instructional Designer with at lea...
7,bs-degree-needed 5-plus-years-experience-needed,Sales and Service Representative SAF-Hollan...
8,bs-degree-needed 2-4-years-experience-needed,"SAF-Holland, Inc. is currently seeking an Inte..."
9,full-time-job hourly-wage,Immediate opening for full time staffing coord...


In [0]:
train = train.fillna('this-does-not-exist')
test = test.fillna("")

In [0]:
train['zero'] = train['tags'].str.contains('this-does-not-exist')
train['licence-needed'] = train['tags'].str.contains('licence-needed')
train['associate-needed'] = train['tags'].str.contains('associate-needed')
train['bs-degree-needed'] = train['tags'].str.contains('bs-degree-needed')
train['ms-or-phd-needed'] = train['tags'].str.contains('ms-or-phd-needed')
train['1-year-experience-needed'] = train['tags'].str.contains('1-year-experience-needed')
train['2-4-years-experience-needed'] = train['tags'].str.contains('2-4-years-experience-needed')
train['5-plus-years-experience-needed'] = train['tags'].str.contains('5-plus-years-experience-needed')
train['supervising-job'] = train['tags'].str.contains('supervising-job')
train['part-time-job'] = train['tags'].str.contains('part-time-job')
train['full-time-job'] = train['tags'].str.contains('full-time-job')
train['hourly-wage'] = train['tags'].str.contains('hourly-wage')
train['salary'] = train['tags'].str.contains('salary')
train.replace((True, False), (1, 0), inplace=True)


train[:5]

,tags,description,zero,licence-needed,associate-needed,bs-degree-needed,ms-or-phd-needed,1-year-experience-needed,2-4-years-experience-needed,5-plus-years-experience-needed,supervising-job,part-time-job,full-time-job,hourly-wage,salary
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...,0,1,0,0,0,0,0,1,1,0,0,0,0
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...,0,0,0,0,0,0,1,0,0,0,1,0,1
2,part-time-job,This is a great position for the right person....,0,0,0,0,0,0,0,0,0,1,0,0,0
3,licence-needed,A large multi-specialty health center is expan...,0,1,0,0,0,0,0,0,0,0,0,0,0
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...,0,0,0,1,0,0,0,1,1,0,1,0,0


Split the training set into a train and development set.

In [0]:
train, dev = train_test_split(train, random_state=233, train_size=0.8)

n_train = train.shape[0]
n_test = test.shape[0]
n_dev = dev.shape[0]

print("Training on", n_train, "examples")
print("Validating on", n_dev, "examples")
print("Testing on", n_test, "examples")

# Concatenate train - dev - test data for easy to handle
full_df = pd.concat([train, dev, test])

Training on 3500 examples
Validating on 875 examples
Testing on 2921 examples


# Time to Tokenize
This turns the text into numerical tokens. Also note that this step also removes all punctuation, rare words, and non words from the data on it's own. Max_words is how many different words/tokens we want to have in our texts.

In [0]:
%%time

Max_words = 36000
print("Transforming text data to sequences...")
raw_text = np.hstack([full_df.description.str.lower()])

print("   Fitting tokenizer...")
tok_raw = Tokenizer(num_words = Max_words,
#                   charaters in this filter string are removed before tokenizing 
                    filters='.!,?"#$%&()*+-/:;<=>@[\\]^_`{|}~\t\n')
tok_raw.fit_on_texts(raw_text)

train = full_df[:n_train]['description'].values
dev = full_df[n_train:n_train+n_dev]['description'].values
test = full_df[n_train+n_dev:]['description'].values

print("   Transforming text to sequences...")
train_tokens = tok_raw.texts_to_sequences(train)
dev_tokens = tok_raw.texts_to_sequences(dev)
test_tokens = tok_raw.texts_to_sequences(test)


Transforming text data to sequences...
   Fitting tokenizer...
   Transforming text to sequences...
CPU times: user 4.63 s, sys: 44 ms, total: 4.68 s
Wall time: 4.57 s


This sets the lengths of all the token arrays to Max_length. Token arrays that have less "words" than Max_length get padded with 0s and token arrays with more "words" get cut short.

In [0]:
Max_length = 700 # 3500
Embed_size = 100

X_train = sequence.pad_sequences(train_tokens, maxlen=Max_length)
X_test = sequence.pad_sequences(test_tokens, maxlen=Max_length)
X_dev = sequence.pad_sequences(dev_tokens, maxlen=Max_length)

Set the Y variables.

In [0]:
train = full_df[:n_train]
dev = full_df[n_train:n_train+n_dev]
test = full_df[n_train+n_dev:]

X_var = ['description']
Y_var = ["zero","licence-needed",	"associate-needed",	"bs-degree-needed",
         "ms-or-phd-needed",	"1-year-experience-needed",	
         "2-4-years-experience-needed","5-plus-years-experience-needed",	
         "supervising-job",	"part-time-job",	"full-time-job",	
         "hourly-wage",	"salary"]

Y_train = train[Y_var].values

Y_dev = dev[Y_var].values

# Put a GLoVe on it!
This puts the glove word vectors into the dataset. As this is a smaller data set and a for fun revist, I used this smaller GLoVe vector than the larger several GB ones. Download the glove vectors here and path the EMBEDDING_FILE to them https://www.kaggle.com/terenceliu4444/glove6b100dtxt/data 

In [0]:
EMBEDDING_FILE = 'glove.6B.100d.txt'

In [0]:
%%time
Embed_size = 100
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

word_index = tok_raw.word_index
nb_words = min(Max_words, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, Embed_size))
for word, i in word_index.items():
    if i >= Max_words: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

del EMBEDDING_FILE
del tok_raw

CPU times: user 15.8 s, sys: 772 ms, total: 16.5 s
Wall time: 16 s


# Custom F1 Metric

In [0]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos


    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos


    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)


    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)


    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))


    return numerator / (denominator + K.epsilon())




def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1score = (2*((precision*recall)/(precision+recall)))
#     if f1score.isnan():
#         f1score = 0.0
    return f1score

# Make the RNN model
Here is the model. It is 2 stacked bidirectional GRU layers followed by some 1d CNN and pooling. It currently is heavier than it needs to be, half these layers will do fine, however due to the smaller dataset size this is managable in Colabs without much trouble or time and makes for improved results.


In [0]:

def get_model():
    inp = Input(shape=(Max_length, ))
    x = Embedding(Max_words, 
                  Embed_size, 
                  weights=[embedding_matrix], 
                  trainable=True)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 7, padding = "valid", kernel_initializer = "he_uniform")(x)
    x = Conv1D(64, kernel_size = 5, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(64, activation="elu")(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation="elu")(x)
    outp = Dense(13, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(
                  loss='binary_crossentropy',
#                   loss=f1,
                  optimizer='adam',
                  metrics=['accuracy', mcor,recall,f1])

    return model

model = get_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 700)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 700, 100)     3600000     input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_2 (SpatialDro (None, 700, 100)     0           embedding_2[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 700, 256)     175872      spatial_dropout1d_2[0][0]        
__________________________________________________________________________________________________
bidirectio

# Callbacks

In [0]:
# Get a checkpoint going
file_path="gdrive/indeed/model_checkpoint.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="val_f1", verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_f1", mode="max", patience=5)


Load models here for more training.

In [0]:
# import keras as K
# model = K.models.load_model("gdrive/indeed/model_checkpoint.hdf5")

# Trian the Model
This block trains the model. The model depth allows it to be trained for a little while before really overfitting.

In [0]:
%%time

model_callbacks = [checkpoint, early]
model.fit(X_train, Y_train,
          batch_size=32,
          epochs=100,
          verbose=2,
          validation_data=(X_dev, Y_dev),
          callbacks = model_callbacks)

# Prediction
This makes the prediction on the test data. Unfortunatly, the Indeed Machine Learning Comp that this was for does not accept late submissions to see what you would have scored after the competition. So I'll have to leave it at that. I have gotten the val_f1 to 0.7365, so should be around there. That is a lot better than my offical score of 0.68580 and would have put me in the top ten. This score could be brought further with other meathods, a stronger word vector, or model ensembling but I will leave this here as this was only for my own learning and curiosity anyways.

In [0]:
%%time
Y_test = model.predict(X_test, batch_size=512)

CPU times: user 7.82 s, sys: 2.68 s, total: 10.5 s
Wall time: 26 s
